# TFIDF Corpus

Reference: https://nlp.stanford.edu/projects/glove/

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from utils.preprocess import getTFIDFCorpus
from utils.utils import getVecForm
from utils.models import simpleNN, simpleLSTM

import tensorflow as tf

In [2]:
PATH = './Datasets/'
DIMS = 300
PREPROCESS = 'tfidf'
MAXVECLEN = 30

## Read GloVe Data

In [3]:
VEC = getTFIDFCorpus(MaxvecLen=MAXVECLEN)
VEC

C:\Users\hites\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(min_df=0.001,
                tokenizer=<function word_tokenize at 0x000002C2FAF44700>)

## Import Dataset

In [4]:
df = pd.read_csv(PATH+'Sarcasm_Headlines_Detection.csv').dropna().reset_index(drop=True)
df

,headline,is_sarcastic
0,versace store clerk sue secret black code mino...,0
1,roseanne revival catch thorny political mood w...,0
2,mom start fear son web series close thing gran...,1
3,boehner want wife listen come alternative debt...,1
4,rowling wish snape happy birthday magical way,0
...,...,...
28608,tyson hold contest let fan submit new idea tor...,1
28609,increasingly cocky bernie sander announce will...,1
28610,cash strap zuckerberg force sell million faceb...,1
28611,grocery store bar actually great little happy ...,1


### Remove excessively long texts

In [5]:
tooLong = []
for i in range(len(df['headline'])):
    if len(df['headline'][i].split()) > MAXVECLEN:
        tooLong.append(i)
for i in tooLong:
    df = df.drop(i, axis=0).reset_index(drop=True)
df

,headline,is_sarcastic
0,versace store clerk sue secret black code mino...,0
1,roseanne revival catch thorny political mood w...,0
2,mom start fear son web series close thing gran...,1
3,boehner want wife listen come alternative debt...,1
4,rowling wish snape happy birthday magical way,0
...,...,...
28607,tyson hold contest let fan submit new idea tor...,1
28608,increasingly cocky bernie sander announce will...,1
28609,cash strap zuckerberg force sell million faceb...,1
28610,grocery store bar actually great little happy ...,1


In [6]:
X_, Y_ = df['headline'], df['is_sarcastic']

## Simple NN

### Convert dataset into Word Vector

In [7]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='sum',
)

In [8]:
X.shape

(28612, 1392)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple NN

In [10]:
snn = simpleNN(X)
snn.fit(x_train, y_train, validation_split=0.3, epochs=5)

Epoch 1/5
470/470 [==============================] - 2s 3ms/step - loss: 0.5833 - accuracy: 0.6843 - val_loss: 0.5673 - val_accuracy: 0.7342
Epoch 2/5
470/470 [==============================] - 1s 3ms/step - loss: 0.4329 - accuracy: 0.7970 - val_loss: 0.5333 - val_accuracy: 0.7335
Epoch 3/5
470/470 [==============================] - 1s 3ms/step - loss: 0.3588 - accuracy: 0.8416 - val_loss: 0.5612 - val_accuracy: 0.7324
Epoch 4/5
470/470 [==============================] - 1s 3ms/step - loss: 0.2787 - accuracy: 0.8918 - val_loss: 0.5851 - val_accuracy: 0.7302
Epoch 5/5
470/470 [==============================] - 1s 3ms/step - loss: 0.2034 - accuracy: 0.9293 - val_loss: 0.6404 - val_accuracy: 0.7251


In [11]:
snn.evaluate(x_test, y_test)

224/224 [==============================] - 0s 1ms/step - loss: 0.6114 - accuracy: 0.7359


[0.6113747954368591, 0.7359150052070618]

## Simple LSTM (sum)

### Convert dataset into Word Vector

In [12]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='sum',
    reshaping=(X.shape[0], 1, X.shape[1])
)

In [13]:
X.shape

(28612, 1, 1392)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple LSTM

In [15]:
lstms = simpleLSTM(X)
lstms.fit(x_train, y_train, validation_split=0.3, epochs=10)

Epoch 1/10
470/470 [==============================] - 4s 4ms/step - loss: 0.6854 - accuracy: 0.5689 - val_loss: 0.6718 - val_accuracy: 0.6025
Epoch 2/10
470/470 [==============================] - 2s 4ms/step - loss: 0.6391 - accuracy: 0.7003 - val_loss: 0.6051 - val_accuracy: 0.7373
Epoch 3/10
470/470 [==============================] - 2s 3ms/step - loss: 0.5617 - accuracy: 0.7551 - val_loss: 0.5435 - val_accuracy: 0.7432
Epoch 4/10
470/470 [==============================] - 2s 3ms/step - loss: 0.5073 - accuracy: 0.7667 - val_loss: 0.5140 - val_accuracy: 0.7467
Epoch 5/10
470/470 [==============================] - 2s 3ms/step - loss: 0.4785 - accuracy: 0.7742 - val_loss: 0.5024 - val_accuracy: 0.7476
Epoch 6/10
470/470 [==============================] - 2s 3ms/step - loss: 0.4629 - accuracy: 0.7770 - val_loss: 0.4990 - val_accuracy: 0.7468
Epoch 7/10
470/470 [==============================] - 2s 4ms/step - loss: 0.4537 - accuracy: 0.7810 - val_loss: 0.4988 - val_accuracy: 0.7476
Epoch 

In [16]:
lstms.evaluate(x_test, y_test)

224/224 [==============================] - 0s 1ms/step - loss: 0.5202 - accuracy: 0.7401


[0.5201503038406372, 0.7401090264320374]

## Simple LSTM (vector)

### Convert dataset into Word Vector

In [17]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec=VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='vector',
    MaxvecLen=MAXVECLEN
)

In [18]:
X.shape

(28612, 30, 1392)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple LSTM

In [20]:
lstmv = simpleLSTM(X)
lstmv.fit(x_train, y_train, validation_split=0.3, epochs=10)

Epoch 1/10
470/470 [==============================] - 6s 10ms/step - loss: 0.6360 - accuracy: 0.6159 - val_loss: 0.5641 - val_accuracy: 0.7257
Epoch 2/10
470/470 [==============================] - 4s 8ms/step - loss: 0.5005 - accuracy: 0.7607 - val_loss: 0.5314 - val_accuracy: 0.7344
Epoch 3/10
470/470 [==============================] - 4s 8ms/step - loss: 0.4670 - accuracy: 0.7869 - val_loss: 0.5231 - val_accuracy: 0.7400
Epoch 4/10
470/470 [==============================] - 4s 8ms/step - loss: 0.4516 - accuracy: 0.7959 - val_loss: 0.5227 - val_accuracy: 0.7429
Epoch 5/10
470/470 [==============================] - 4s 8ms/step - loss: 0.4413 - accuracy: 0.7999 - val_loss: 0.5273 - val_accuracy: 0.7429
Epoch 6/10
470/470 [==============================] - 4s 8ms/step - loss: 0.4320 - accuracy: 0.8037 - val_loss: 0.5338 - val_accuracy: 0.7415
Epoch 7/10
470/470 [==============================] - 4s 8ms/step - loss: 0.4221 - accuracy: 0.8068 - val_loss: 0.5465 - val_accuracy: 0.7380
Epoch

In [21]:
lstmv.evaluate(x_test, y_test)

224/224 [==============================] - 1s 4ms/step - loss: 0.5453 - accuracy: 0.7382


[0.545265257358551, 0.7381518483161926]